In [101]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, './../src')

from data_loader import load_compressed_rept_data
from energy_channels import EnergyChannel
import datetime
import matplotlib.pyplot as plt
import plot_tools
import numpy as np
import pandas as pd

import importlib
importlib.reload(plot_tools)

%matplotlib qt

In [102]:
start = datetime.datetime(year = 2013, month = 1, day = 1)
end = datetime.datetime(year = 2014, month = 1, day = 1)
satellite = "A"

data = load_compressed_rept_data(satellite=satellite, start=start, end=end)


Loading REPT data between: 2013-01-01 00:00:00 and 2014-01-01 00:00:00.
Loading : REPT_201301_A.npz
Loading : REPT_201302_A.npz
Loading : REPT_201303_A.npz
Loading : REPT_201304_A.npz
Loading : REPT_201305_A.npz
Loading : REPT_201306_A.npz
Loading : REPT_201307_A.npz
Loading : REPT_201308_A.npz
Loading : REPT_201309_A.npz
Loading : REPT_201310_A.npz
Loading : REPT_201311_A.npz
Loading : REPT_201312_A.npz
Loading : REPT_201401_A.npz


In [103]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax[0].set_xlim(start, end)

ax[1].set_xlim(start, end)
FESA = plot_tools.plot_L_CUT(refs=data, L_CUT=3.5, axis=ax[1], grid_t="9h", avg_t="12h")

diff = FESA.pct_change(periods=1)

colors = plt.cm.plasma(np.linspace(0, 1, 7))

for column in diff.columns:
    ax[0].plot(diff.index, diff[column], color=colors[column])
    ax[0].set_yscale('symlog')
    ax[0].hlines(y = 1, xmin=start, xmax=end, color="black")


print(FESA)
print(diff[0])

fig.savefig("Test.pdf", format="pdf")


C:\Users\brofe\AppData\Local\Temp\ipykernel_29792\2685777435.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  diff = FESA.pct_change(periods=1)


                               0            1            2            3  \
2013-01-01 00:00:00  5835.132904  5686.305340  4551.987667  3509.606938   
2013-01-01 09:00:00  6169.656280  6208.357419  4782.577202  3685.205190   
2013-01-01 18:00:00  6189.168206  6254.387429  4778.235863  3592.959323   
2013-01-02 03:00:00  6325.706350  6211.331739  4914.429756  3716.722171   
2013-01-02 12:00:00  6074.230985  6012.417128  4764.989218  3599.313131   
...                          ...          ...          ...          ...   
2013-12-30 09:00:00   170.175347   175.089654   125.560151    49.604479   
2013-12-30 18:00:00   164.061726   156.247296   132.254083    54.373630   
2013-12-31 03:00:00   185.587028   182.653637   141.141170    57.873092   
2013-12-31 12:00:00   165.070391   144.645301   136.193047    54.817203   
2013-12-31 21:00:00   152.954350   161.109257   122.872231    63.433454   

                               4           5           6  
2013-01-01 00:00:00  1688.015188  712.18